In [ ]:
class SurveyResults(Base):
    __tablename__ = "survey_results"
    id = Column(Integer, primary_key=True)
    signature = Column(String)
    waypoint_symbol = Column(String)
    deposits = Column(String)  # actually a list
    expiration = Column(String)  # this is a datetime but i'm not sure how to handle it before it goes into postgresql
    # datetime.fromisoformat(
    size = Column(String)  # actually an enum i think?
    time_stamp = Column('timestamp', TIMESTAMP(timezone=False), nullable=False, default=sa.func.now())

In [ ]:
import json
# import urllib2
from urllib import request
from playhouse.postgres_ext import *

db = PostgresqlExtDatabase('my_database')

class APIResponse(Model):
    url = CharField()
    response = JSONField()

    class Meta:
        database = db

    @classmethod
    def request(cls, url):
        fh = urllib.urlopen(url)
        return cls.create(url=url, response=json.loads(fh.read()))

APIResponse.create_table()

# Store a JSON response.
offense = APIResponse.request('http://crime-api.com/api/offense/')
booking = APIResponse.request('http://crime-api.com/api/booking/')

# Query a JSON data structure using a nested key lookup:
offense_responses = APIResponse.select().where(
    APIResponse.response['meta']['model'] == 'offense')

# Retrieve a sub-key for each APIResponse. By calling .as_json(), the
# data at the sub-key will be returned as Python objects (dicts, lists,
# etc) instead of serialized JSON.
q = (APIResponse
     .select(
       APIResponse.data['booking']['person'].as_json().alias('person'))
     .where(APIResponse.data['meta']['model'] == 'booking'))

for result in q:
    print(result.person['name'], result.person['dob'])

In [ ]:
from playhouse.postgres_ext import ArrayField
import peewee
import datetime
from peewee import *
from SECRETS import sql_account, sql_pw
import logging

logger = logging.getLogger('peewee')
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.DEBUG)

pg_db = peewee.PostgresqlDatabase('spacetraders',
                                    user=sql_account,
                                    password=sql_pw,
                                    host='localhost',
                                    port=5432)

class BaseModel(Model):
    class Meta:
        database = pg_db

class SurveyResult(BaseModel):
    signature = TextField
    waypoint_symbol = TextField
    expiration = DateTimeField
    size = TextField
    time_stamp = DateTimeField

class Deposit(BaseModel):
    signature = ForeignKeyField(SurveyResult, backref='deposits')
    survey_deposits = ArrayField(CharField)

In [12]:
def populate_test_data():
    pg_db.create_tables([User, Tweet, Favorite])

    data = (
        ('huey', ('meow', 'hiss', 'purr')),
        ('mickey', ('woof', 'whine')),
        ('zaizee', ()))
    for username, tweets in data:
        user = User.create(username=username)
        for tweet in tweets:
            Tweet.create(user=user, content=tweet)

    # Populate a few favorites for our users, such that:
    favorite_data = (
        ('huey', ['whine']),
        ('mickey', ['purr']),
        ('zaizee', ['meow', 'purr']))
    for username, favorites in favorite_data:
        user = User.get(User.username == username)
        for content in favorites:
            tweet = Tweet.get(Tweet.content == content)
            Favorite.create(user=user, tweet=tweet)

populate_test_data()

('CREATE TABLE IF NOT EXISTS "user" ("id" SERIAL NOT NULL PRIMARY KEY, "username" TEXT NOT NULL)', [])
('CREATE TABLE IF NOT EXISTS "user" ("id" SERIAL NOT NULL PRIMARY KEY, "username" TEXT NOT NULL)', [])
('CREATE TABLE IF NOT EXISTS "tweet" ("id" SERIAL NOT NULL PRIMARY KEY, "content" TEXT NOT NULL, "timestamp" TIMESTAMP NOT NULL, "user_id" INTEGER NOT NULL, FOREIGN KEY ("user_id") REFERENCES "user" ("id"))', [])
('CREATE TABLE IF NOT EXISTS "tweet" ("id" SERIAL NOT NULL PRIMARY KEY, "content" TEXT NOT NULL, "timestamp" TIMESTAMP NOT NULL, "user_id" INTEGER NOT NULL, FOREIGN KEY ("user_id") REFERENCES "user" ("id"))', [])
('CREATE INDEX IF NOT EXISTS "tweet_user_id" ON "tweet" ("user_id")', [])
('CREATE INDEX IF NOT EXISTS "tweet_user_id" ON "tweet" ("user_id")', [])
('CREATE TABLE IF NOT EXISTS "favorite" ("id" SERIAL NOT NULL PRIMARY KEY, "user_id" INTEGER NOT NULL, "tweet_id" INTEGER NOT NULL, FOREIGN KEY ("user_id") REFERENCES "user" ("id"), FOREIGN KEY ("tweet_id") REFERENCES "t

IntegrityError: duplicate key value violates unique constraint "user_username"
DETAIL:  Key (username)=(huey) already exists.


In [13]:
import peewee
import datetime
from peewee import *
from SECRETS import sql_account, sql_pw
import logging

logger = logging.getLogger('peewee')
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.DEBUG)

pg_db = peewee.PostgresqlDatabase('spacetraders',
                                    user=sql_account,
                                    password=sql_pw,
                                    host='localhost',
                                    port=5432)
class BaseModel(Model):
    class Meta:
        database = pg_db

class User(BaseModel):
    username = TextField()

class Tweet(BaseModel):
    content = TextField()
    timestamp = DateTimeField(default=datetime.datetime.now)
    user = ForeignKeyField(User, backref='tweets')

class Favorite(BaseModel):
    user = ForeignKeyField(User, backref='favorites')
    tweet = ForeignKeyField(Tweet, backref='favorites')




In [14]:
def populate_test_data():
    pg_db.create_tables([User, Tweet, Favorite])

    data = (
        ('huey', ('meow', 'hiss', 'purr')),
        ('mickey', ('woof', 'whine')),
        ('zaizee', ()))
    for username, tweets in data:
        user = User.create(username=username)
        for tweet in tweets:
            Tweet.create(user=user, content=tweet)

    # Populate a few favorites for our users, such that:
    favorite_data = (
        ('huey', ['whine']),
        ('mickey', ['purr']),
        ('zaizee', ['meow', 'purr']))
    for username, favorites in favorite_data:
        user = User.get(User.username == username)
        for content in favorites:
            tweet = Tweet.get(Tweet.content == content)
            Favorite.create(user=user, tweet=tweet)

populate_test_data()

('SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = %s ORDER BY tablename', ('public',))
('SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = %s ORDER BY tablename', ('public',))
('SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = %s ORDER BY tablename', ('public',))
('CREATE TABLE IF NOT EXISTS "tweet" ("id" SERIAL NOT NULL PRIMARY KEY, "content" TEXT NOT NULL, "timestamp" TIMESTAMP NOT NULL, "user_id" INTEGER NOT NULL, FOREIGN KEY ("user_id") REFERENCES "user" ("id"))', [])
('CREATE TABLE IF NOT EXISTS "tweet" ("id" SERIAL NOT NULL PRIMARY KEY, "content" TEXT NOT NULL, "timestamp" TIMESTAMP NOT NULL, "user_id" INTEGER NOT NULL, FOREIGN KEY ("user_id") REFERENCES "user" ("id"))', [])
('CREATE TABLE IF NOT EXISTS "tweet" ("id" SERIAL NOT NULL PRIMARY KEY, "content" TEXT NOT NULL, "timestamp" TIMESTAMP NOT NULL, "user_id" INTEGER NOT NULL, FOREIGN KEY ("user_id") REFERENCES "user" ("id"))', [])
('CREATE INDEX IF NOT EXISTS "tweet_user_id" ON "tweet" ("

IntegrityError: duplicate key value violates unique constraint "user_username"
DETAIL:  Key (username)=(huey) already exists.


In [19]:
query = Tweet.select().join(User)
for tweet in query:
    print(tweet.user.username, tweet.content)


('SELECT "t1"."id", "t1"."content", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1" INNER JOIN "user" AS "t2" ON ("t1"."user_id" = "t2"."id")', [])
('SELECT "t1"."id", "t1"."content", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1" INNER JOIN "user" AS "t2" ON ("t1"."user_id" = "t2"."id")', [])
('SELECT "t1"."id", "t1"."content", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1" INNER JOIN "user" AS "t2" ON ("t1"."user_id" = "t2"."id")', [])
('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."id" = %s) LIMIT %s OFFSET %s', [1, 1, 0])
('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."id" = %s) LIMIT %s OFFSET %s', [1, 1, 0])
('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."id" = %s) LIMIT %s OFFSET %s', [1, 1, 0])
('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."id" = %s) LIMIT %s OFFSET %s', [1, 1, 0])
('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."id" = %s) LIMIT %s OFFSET

huey meow
huey hiss
huey purr
mickey woof
mickey whine


In [18]:
query = Tweet.select().join(User).where(User.username == 'huey')
for tweet in query:
    print(tweet.user.username, tweet.content)

('SELECT "t1"."id", "t1"."content", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1" INNER JOIN "user" AS "t2" ON ("t1"."user_id" = "t2"."id") WHERE ("t2"."username" = %s)', ['huey'])
('SELECT "t1"."id", "t1"."content", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1" INNER JOIN "user" AS "t2" ON ("t1"."user_id" = "t2"."id") WHERE ("t2"."username" = %s)', ['huey'])
('SELECT "t1"."id", "t1"."content", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1" INNER JOIN "user" AS "t2" ON ("t1"."user_id" = "t2"."id") WHERE ("t2"."username" = %s)', ['huey'])
('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."id" = %s) LIMIT %s OFFSET %s', [1, 1, 0])
('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."id" = %s) LIMIT %s OFFSET %s', [1, 1, 0])
('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."id" = %s) LIMIT %s OFFSET %s', [1, 1, 0])
('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."id" = %s) LIMIT %s OFFSET %s', 

huey meow
huey hiss
huey purr


In [8]:
query = (Tweet
         .select()
         .join(User, on=(Tweet.user == User.id))
         .where(User.username == 'huey'))
for tweet in query:
    print(tweet.content)

('SELECT "t1"."id", "t1"."content", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1" INNER JOIN "user" AS "t2" ON ("t1"."user_id" = "t2"."id") WHERE ("t2"."username" = %s)', ['huey'])


meow
hiss
purr


In [9]:
huey = User.get(User.username == 'huey')
for tweet in huey.tweets:
    print(tweet.content)



('SELECT "t1"."id", "t1"."username" FROM "user" AS "t1" WHERE ("t1"."username" = %s) LIMIT %s OFFSET %s', ['huey', 1, 0])
('SELECT "t1"."id", "t1"."content", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1" WHERE ("t1"."user_id" = %s)', [1])


meow
hiss
purr


In [10]:
huey.tweets.sql()

('SELECT "t1"."id", "t1"."content", "t1"."timestamp", "t1"."user_id" FROM "tweet" AS "t1" WHERE ("t1"."user_id" = %s)',
 [1])